## 选修内容


### 神经网络的梯度怎么算

Chain Rule:

假设 $L(w)=f(g(h(w)))$

那么 $L'(w)=f'(g(h(w))) \cdot g'(h(w)) \cdot h'(w)$

<img src="backprop.png" style="margin-left: 0px" width="600px">

蓝色的过程叫 Forward Pass，红色的过程叫 Backward Pass，整个过程叫 Backpropagation


### 几个常用的超参

### 1、过拟合与欠拟合

<br />
<img src="overfit.png" style="margin-left: 0px" width="800px">

<div class="alert alert-success">
<b>奥卡姆剃刀：</b> 两个处于竞争地位的理论能得出同样的结论，那么简单的那个更好。
</div>

**防止过拟合的方法（1）：**Weight Decay

$J(\omega)=L(D,\omega)+\lambda\|\omega\| \Rightarrow \nabla_{\omega}J=\nabla_{\omega}L + \frac{1}{2}\lambda\omega$

- 惩罚参数的复杂性（$L_2$-norm）：等价与在梯度上减去参数本身（乘一个小数作为权重）
- Weight Decay 就是前面那个权重$\lambda$

**防止过拟合的方法（2）：**Dropout

- 我们在前向传播的时候，概率性的（临时）删除一部分神经元，这样可以使模型泛化性更强，因为它不会太依赖某些局部的特征
- 这样训练$N$次，等价于训练$N$不同的网络，再取平均值；$N$个网络不会同时过拟合于与一个结果，这样平均值的方式能有效减少过拟合的干扰。

<img src="dropout.jfif" style="margin-left: 0px" width="400px">

### 学习率调整策略

- 开始时学习率大些：快速到达最优解附近
- 逐渐减小学习率：避免跳过最优解
- NLP 任务的损失函数有很多“悬崖峭壁”，自适应学习率更能处理这种极端情况，避免梯度爆炸。

<img src="scheduler.png" style="margin-left: 0px" width="400px">

几种常用的学习率调整器

<img src="lr_scheduler.jpg" style="margin-left: 0px" width="400px">

**防止过拟合的方法（3）：**学习率 Warm Up

先从一个很小的学习率逐渐上升到正常学习率，在稳步减小学习率

- 其原理尚未被充分证明
- 经验主义解释：减缓模型在初始阶段对 mini-batch 的提前过拟合现象，保持分布的平稳
- 经验主义解释：有助于保持模型深层的稳定性

<img src="warmup.png" style="margin-left: 0px" width="400px">

<div class="alert alert-success">
<b>应用场景：</b> (1) 当网络非常容易nan时候；(2) 如果训练集损失很低，准确率高，但测试集损失大，准确率低. 
</div>


### 自然语言处理常见的网络结构

<div class="alert alert-warning">
<b>思考：</b> 图像天生可以表示成矩阵（或tensor），那文本怎么表示成矩阵（或tensor）
</div>


### 1、文本卷积神经网络 TextCNN

<br />

一个窗口的卷积和 Pooling 过程

<img src="conv_maxpooling_steps.gif" style="margin-left: 0px" width="600px">

不同大小的窗口分别做卷积和 Pooling，结果拼在一起

<img src="TextCNN.jpg" style="margin-left: 0px" width="600px">

- 参数量较少、好训练、算力要求低
- 适合文本分类问题
- 善于表示局部特征（卷积窗口），不擅长表示长上下文依赖关系

### 2、循环神经网络 RNN

<br />

首先：输入是一个序列

<img src="RNN.png" style="margin-left: 0px" width="600px">

但这种简易 RNN 有很多问题，最大问题是随着序列长度增加，梯度消失或爆炸

<img src="LSTM.png" style="margin-left: 0px" width="600px">

LSTM 和 GRU 通过「门」来控制上文的状态被记住还是遗忘，同时防止梯度消失或爆炸

以 LSTM 为例：

<img src="lstm.jfif" style="margin-left: 0px" width="600px">

### 3、Attention (for RNN)

<br />

给定当前的输入，上文的一些信息比另一些重要

<img src="attention.gif" style="margin-left: 0px" width="600px">

<br />

<font color='blue'>于是设计一个可微的函数就可以把它加入到网络中来试试，反正也没有全局最优解</font>

<br />
<img src="attention-fn.png" style="margin-left: 0px" width="600px">

- 对当前 token $t$的上文的每个 token $i$计算上述 score
- 将这些 score 做 softmax 得到权重$\alpha_{t,i}$
- 将上文的隐层状态乘以其权重并相加$c_t=\sum_i\alpha_{t,i}h_i$
- 将$c_t$与当前 token 的状态拼接在一起$s_t=\mathrm{concat}(h_t,c_t)$
- 激活输出$y_t=\sigma(s_t)$


### Transformer 江山一统

<div class="alert alert-warning">
<b>思考：</b> RNN有什么缺点？大模型为什么不是很多层RNN？
</div>

<br />
<img src="transformer.gif" style="margin-left: 0px" width="800px">

### 1、**消除恐惧：**我们亲手写一个 Transformer

#### 1.1、Embeddings


In [ ]:
import torch.nn as nn
import torch


class PositionalEmbedding(nn.Module):

    def __init__(self, embed_size, max_len=512):
        super().__init__()

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, embed_size).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, embed_size, 2).float()
                    * -(math.log(10000.0) / embed_size)).exp()

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]


class BERTEmbedding(nn.Module):

    def __init__(self, vocab_size, embed_size, dropout=0.1):
        """
        :param vocab_size: 词表大小
        :param embed_size: embedding维度768
        :param dropout: dropout概率
        """
        super().__init__()
        self.token_embedding = nn.Embedding(
            vocab_size, embed_size, padding_idx=0)
        self.position_embedding = PositionalEmbedding(
            embed_size=embed_size, max_len=512)
        self.token_type_embedding = nn.Embedding(2, embed_size, padding_idx=0)
        self.dropout = nn.Dropout(p=dropout)
        self.embed_size = embed_size

    def forward(self, input_ids, token_type_ids):
        x = self.token_embedding(input_ids) + self.position_embedding(
            input_ids) + self.token_type_embedding(token_type_ids)
        return self.dropout(x)

#### 1.2、单头 Attention


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch

import math

'''
query = query_linear(x)
key = key_linear(x)
value = value_linear(x)
'''

# 单个头的注意力计算


class Attention(nn.Module):

    def forward(self, query, key, value, mask=None, dropout=None):
        scores = torch.matmul(query, key.transpose(-2, -1)) \
            / math.sqrt(query.size(-1))

        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        p_attn = F.softmax(scores, dim=-1)

        if dropout is not None:
            p_attn = dropout(p_attn)

        return torch.matmul(p_attn, value), p_attn

<br />
每个token对应的query向量与每个token对应的key向量做内积

<img src="kq2.gif" style="margin-left: 0px" width="800px">

<br />
将上述内积取softmax（得到0~1之间的值，即为attention权重）

<img src="kq_softmax.gif" style="margin-left: 0px" width="800px">

<br />
计算每个token相对于所有其它token的attention权重（最终构成一个$L\times L$的attention矩阵）

<img src="kq_softmax2.gif" style="margin-left: 0px" width="800px">

<br />
每个token对应的value向量乘以attention权重，并相加，得到当前token的self-attention value向量

<img src="v.gif" style="margin-left: 0px" width="800px">

<br />
将上述操作应用于每个token
<img src="v2.gif" style="margin-left: 0px" width="800px">

<br />
以上是一个头的操作，同时（并行）应用于多个独立的头


#### 1.3、多头 Attention

将每个头得到向量拼接在一起，最后乘一个线性矩阵，得到 multi-head attention 的输出

<img src="multi-head.gif" style="margin-left: 0px" width="800px">


In [ ]:
import torch.nn


class MultiHeadedAttention(nn.Module):
    def __init__(self, head_num, hidden_size, dropout=0.1):
        """
        :param head_num: 头的个数，必须能被hidden_size整除
        :param hidden_size: 隐层的维度，与embed_size一致
        """
        super().__init__()
        assert hidden_size % head_num == 0

        self.per_head_dim = hidden_size // head_num
        self.head_num = head_num
        self.hidden_size = hidden_size

        self.query_linear = nn.Linear(hidden_size, hidden_size)
        self.key_linear = nn.Linear(hidden_size, hidden_size)
        self.value_linear = nn.Linear(hidden_size, hidden_size)

        self.output_linear = nn.Linear(hidden_size, hidden_size)
        self.attention = Attention()

        self.dropout = nn.Dropout(p=dropout)

    def reshape(self, x, batch_size):
        # 拆成多个头
        return x.view(batch_size, -1, self.head_num, self.per_head_dim).transpose(1, 2)

    def forward(self, x, mask=None):
        batch_size = x.size(0)

        query = self.reshape(self.query_linear(x))
        key = self.reshape(self.key_linear(x))
        value = self.reshape(self.value_linear(x))

        # 每个头计算attention
        x, attn = self.attention(
            query, key, value, mask=mask, dropout=self.dropout
        )

        # 把每个头的attention*value拼接在一起
        x = x.transpose(1, 2).contiguous().view(
            batch_size, -1, self.hidden_size)

        # 乘一个线性矩阵
        return self.output_linear(x)

#### 1.4、全连接网络（Feed-Forward Network）


In [ ]:
import torch.nn as nn


class FeedForward(nn.Module):

    def __init__(self, hidden_size, dropout=0.1):
        super(FeedForward, self).__init__()
        self.input_layer = nn.Linear(hidden_size, hidden_size*4)
        self.output_layer = nn.Linear(hidden_size*4, hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.GELU()

    def forward(self, x):
        x = self.input_layer(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.output_layer(x)
        return x

#### 1.5、拼成一层 Transformer


In [ ]:
import torch.nn as nn


class TransformerBlock(nn.Module):

    def __init__(self, hidden_size, head_num, dropout=0.1):
        super().__init__()
        self.multi_head_attention = MultiHeadedAttention(head_num, hidden_size)
        self.feed_forward = FeedForward(hidden_size, dropout=dropout)
        self.layer_norm1 = nn.LayerNorm(hidden_size)
        self.dropout1 = nn.Dropout(dropout)
        self.layer_norm2 = nn.LayerNorm(hidden_size)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, mask):
        x0 = x
        # 多头注意力层
        x = self.multi_head_attention(x, mask)

        # 残差和LayerNorm层(1)
        x = self.dropout1(x)
        x = self.layer_norm1(x0+x)

        # 前向网络层
        x1 = x
        x = self.feed_forward(x)

        # 残差和LayerNorm层(2)
        x = self.dropout2(x)
        x = self.layer_norm2(x1+x)
        return x

<br />
Multi-head attention的输出，经过残差和norm之后进入一个两层全连接网络
<img src="ffn.gif" style="margin-left: 0px" width="800px">


Layernorm:

$y=\frac{x-\mathrm{E}(x)}{\sqrt{\mathrm{Var}(x)+\epsilon}}*\gamma+\beta$

其中 $\gamma$ 和 $\beta$ 是可训练的参数，$\epsilon=10^{-5}$是超参，保持数值稳定性。


#### 1.6、多层 Transformer 构成 BERT Encoder


In [ ]:
import torch.nn as nn


class BERT(nn.Module):

    def __init__(self, vocab_size, hidden_size=768, layer_num=12, head_num=12, dropout=0.1):

        super().__init__()
        # Embedding层
        self.embedding = BERTEmbedding(
            vocab_size=vocab_size, embed_size=hidden_size)
        # N层Transformers
        self.transformer_blocks = nn.ModuleList(
            [TransformerBlock(hidden_size, head_num, dropout)
             for _ in range(layer_num)]
        )

    def forward(self, input_ids, token_type_ids):
        """
        tokenizer(["你好吗","你好"], text_pair=["我很好","我好"], max_length=10, padding='max_length',truncation=True)
        [CLS]你好吗[SEP]我很好[SEP][PAD]
        [CLS]你好[SEP]我好[SEP][PAD][PAD][PAD]  
        input_ids: [
            [101, 872, 1962, 1408, 102, 2769, 2523, 1962, 102, 0],
            [101, 872, 1962, 102, 2769, 1962, 102, 0, 0, 0]
        ]
        token_type_ids：[
                [0, 0, 0, 0, 0, 1, 1, 1, 1, 0],
                [0, 0, 0, 0, 1, 1, 1, 0, 0, 0]
            ]
        """
        attention_mask = (x > 0).unsqueeze(
            1).repeat(1, x.size(1), 1).unsqueeze(1)

        # 计算embedding
        x = self.embedding(input_ids, token_type_ids)

        # 逐层代入Tranformers
        for transformer in self.transformer_blocks:
            x = transformer.forward(x, attention_mask)

        return x

### 2、Transformer 怎么用

#### 2.1、 Encoder-Only LM 用于文本表示

针对不同下游任务，在 Encoder 上面添加不同的输出层
<br />
<img src="BERT.png" style="margin-left: 0px" width="600px">

#### 2.2、 Encoder上加头，面向不同任务

#### Linear Head文本分类

<img src="bert-classification.png" style="margin-left: 0px" width="600px">

#### BERT-BiLSTM-CRF一个序列标注的经典网络结构

<img src="bert-bilstm-crf.jpg" style="margin-left: 0px" width="600px">

#### 2.3、 Encoder-Decoder LM，机器翻译/文本生成（大语言模型的一种形态）

- Decoder 也是 N 层 transformer 结构
- 生成一个 token，把它加入上文，再生成下一个 token，以此类推
  <br />
  <img src="decoder1.png" style="margin-left: 0px" width="600px">

1. Decoder 的每个 token 与 decoder 上文的 token 做一次 attention

2. 输出 add & norm 之后再与 encoder 最后一层的输出做一次 sttention

<br />
<img src="decoder2.png" style="margin-left: 0px" width="600px">

<div class="alert alert-warning">
<b>注意：</b> Decoder 的 token 只能 attend 到上文的 token（因为此时下文还没有出现）
</div>

<br />
<img src="decoder3.png" style="margin-left: 0px" width="600px">


#### 2.4、Decoder-Only LM 也叫 Causal LM 或 Left-to-right LM（GPT 家族）

<br />
<img src="decoder4.png" style="margin-left: 0px" width="400px">

#### 2.5、 大语言模型族谱

<br />
<img src="llm.jpg" style="margin-left: 0px" width="800px">
